In [ ]:
!pip install pandas matplotlib seaborn

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
path_confirmed = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
                  "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
path_death = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
              "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv") 
path_recovered = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
                  "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

In [3]:
def set_index(df):
    # Set region, country, lat and long as index
    index = pd.MultiIndex.from_frame(df.iloc[:, :4])
    # Set the index and transpose
    df = df.iloc[:, 4:].set_index(index).T
    # Set date as index
    return df.set_index(pd.to_datetime(df.index, dayfirst=False))

df_conf = pd.read_csv(path_confirmed)
df_conf = set_index(df_conf)

In [16]:
import matplotlib.dates as mdates


def my_widget(df_input, figsize=(20,5)):
    df_input[(None, 'Total', None, None)] = df_input.groupby(level=1, axis=1).sum().sum(axis=1)
        
    countries = ['No country'] + df_input.columns.get_level_values(1).unique().sort_values().tolist()
    
    @interact(country_a=countries, 
              country_b=countries, 
              country_c=countries, 
              start_date=df_conf.index, 
              end_date=df_conf.index,
              daily_var=False)
    def plot_range(country_a, 
                   country_b, 
                   country_c, 
                   start_date, 
                   end_date,
                   daily_var):
        df = (
            df_input
            .groupby(level=1, axis=1).sum()
            .loc[lambda df: (df.index >= start_date) &  (df.index <= end_date)]
        )

        if daily_var:
            df -= df.shift()
        
        df = (
            df
            .loc[:, lambda df: df.columns.get_level_values('Country/Region').isin([country_a, 
                                                                                   country_b, 
                                                                                   country_c])]
            .reset_index()
            .melt(id_vars='index')
            .rename(columns={"index": "Date", "value": "Count"})
        )

    
        plt.figure(figsize=figsize)

        if daily_var:
            ax = sns.barplot(x="Date", y="Count", hue="Country/Region", data=df)
        else:
            ax = sns.lineplot(x="Date", y="Count", hue="Country/Region", data=df)

        ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
        ax.grid(True)
        plt.xticks(rotation=45);

my_widget(df_conf.rename(columns={'Others': 'Diamond Princess'}))

interactive(children=(Dropdown(description='country_a', options=('No country', 'Afghanistan', 'Albania', 'Alge…